In [25]:
import mediapipe as mp
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

In [26]:
mp_holistic = mp.solutions.holistic    
mp_drawing = mp.solutions.drawing_utils 
mp_face_mesh = mp.solutions.face_mesh 

In [27]:
# Images will be provided from opencv in the form of Frames
# Why to convert from BGR to RGB : If the conversion is skipped, MediaPipe might produce incorrect outputs, as its models and pipelines # # are trained and optimized for RGB images.

def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)  # color conversion from bgr to rgb
    image.flags.writeable = False                   # sets the image writeable status to false 
    results = model.process(image)               # Image is actually detected here using mediapipe 
    image.flags.writeable = True                    # Again the image is set it writeable
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # color conversion from rgb to bgr
    return image,results
    

In [28]:
def draw_landmarks(image,results):
    
    # Here we are making a customized drawing lines 
    landmark_drawing_spec = mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1)
    connection_drawing_spec = mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
    
    landmark_drawing_spec1 = mp_drawing.DrawingSpec(color=(466,466,466), thickness=1, circle_radius=4)
    connection_drawing_spec1 = mp_drawing.DrawingSpec(color=(000, 000, 000), thickness=1, circle_radius=2)
   
    
    # This will draw the face ,pose and hands connection
    
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_face_mesh.FACEMESH_CONTOURS,
                              landmark_drawing_spec,connection_drawing_spec)
    
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              landmark_drawing_spec1,connection_drawing_spec1)
    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              landmark_drawing_spec1,connection_drawing_spec1)
    
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                               landmark_drawing_spec1,connection_drawing_spec1)

In [29]:
colors = [(245,117,16),(117,245,16),(16,117,245)]
def prob_vis(res1,actions,input_frame,colors):
    output_frame = input_frame.copy()
    for num ,prob in enumerate(res1):
        cv2.rectangle(output_frame,(0,60+num*40),(int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

In [32]:
import cv2
import numpy as np
import mediapipe as mp
import os
from tensorflow.keras.models import load_model

# Paths
data_path = "F:\\Siddhant\\Projects\\Sign_Language_Detection\\MP_DATA"  # Directory where .npy files are stored
model_path = "F:\\Siddhant\\Projects\\Sign_Language_Detection\\action.h5"

# Load Model
model = load_model(model_path)

# Define actions and colors
actions = ["hello", "thanks", "yes", "no"]  # Define your actions here
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (16, 245, 117)]

# Prediction variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

# Mediapipe setup
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Function to get keypoints for an action dynamically
def load_action_keypoints(action, sequence_num):
    keypoints_path = os.path.join(data_path, action, f"{sequence_num}.npy")
    if os.path.exists(keypoints_path):
        return np.load(keypoints_path)
    else:
        print(f"File not found: {keypoints_path}")
        return None

# Function to visualize probabilities
def prob_vis(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

# Video capture
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        res, frame = cap.read()

        # Using a function to make detection
        image, results = mediapipe_detection(frame, holistic)

        # Draw landmarks in real time
        draw_landmarks(image, results)

        # Dynamically load keypoints for all actions
        keypoints = []
        for action in actions:
            for sequence_num in range(30):  # Adjust sequence range as per your data
                kp = load_action_keypoints(action, sequence_num)
                if kp is not None:
                    keypoints.append(kp)
        
        if len(keypoints) > 0:
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                res1 = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res1)])
                predictions.append(np.argmax(res1))

            # Visual logic
            if len(predictions) >= 10 and np.unique(predictions[-10:])[0] == np.argmax(res1):
                if res1[np.argmax(res1)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res1)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res1)])
                    else:
                        sentence.append(actions[np.argmax(res1)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            image = prob_vis(res1, actions, image, colors)

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Showing to screen
        cv2.imshow('Opencv_video', image)

        # Breaking loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\0.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\1.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\2.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\3.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\4.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\5.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\6.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\7.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\8.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\9.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\10.npy
File not found: F:\Siddhant\Projects\Sign_Language_Detection\MP_DATA\hello\11.npy
File not found: F:\Siddhan